# Introduction
This notebook shows how to submit a job with current `Jupyter` notebook. To authenticated by `OpenPAI`, please store your system information in ***openpai.json*** (`OpenPAI` configuration file), including key information like **pai_uri**, **user**, **passwd** and **hdfs_web_uri**

Note:
- **pai_uri** is in the format of `http://x.x.x.x`
- **hdfs_web_uri** is in the format of `http://x.x.x.x:port`
- ***Keep your configuration secret. Never commit it***

# Prepare environment

In [ ]:
! pip install -U https://codeload.github.com/mydmdm/paiclient/zip/master

# Do whatever you want

In [ ]:
# set logging to console and file
import logging
log_file = 'log.txt'

logger = logging.getLogger(__name__)
logger.setLevel(level = logging.INFO)

handler = logging.FileHandler(log_file, mode='w')
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)


In [ ]:
from paiclient.core import in_job_container, in_debug_mode

if in_job_container():
    logger.info('I am in a container')
else:
    logger.info('I am out of a container')

In [ ]:
# upload necessary files
from paiclient.core import Client
import os

if in_job_container() and not in_debug_mode():
    client = Client.from_envs()
    client.storage.upload(local_path=log_file, remote_path='{}/output/{}'.format(os.environ['PAISDK_JOB_DIR'], log_file))

# Submit to `OpenPAI`
After debugging in your environment, just submit current notebook and let it run.

In [ ]:
from paiclient.notebook import submit_notebook
from paiclient.core import Client
from uuid import uuid4

if not in_job_container() or in_debug_mode():
    client = Client.from_json('openpai.json')
    _, name, _ = parse_notebook_path()
    job_name = '{}_{}'.format(name, uuid4().hex)
    job_dir = '/user/{}/jobs/{}'.format(client.user, job_name)
    submit_notebook(
        client, job_name,
        image='jupyter/minimal-notebook', # 'ubuntu:16.04'
        job_dir=job_dir
    )
    logger.info(client.get_job_link(job_name))
